In [85]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

import os
import scipy
import sqlite3
from collections import defaultdict
import matplotlib.pyplot as plt


In [10]:
conn = sqlite3.connect(os.path.realpath('') + "/lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 200000);", conn)

# Separate loans that defaulted and were paid off
df_default = pd.read_sql_query( "select * from loan where loan_status='Charged Off' OR loan_status='Default' OR loan_status='Late (31-120 days)' OR loan_status='Late (16-30 days)' OR loan_status='Does not meet the credit policy. Status:Charged Off' AND id in(select id from loan order by random());", conn)
df_fully_paid = pd.read_sql_query("select * from loan where loan_status='Issued' OR loan_status='Fully Paid' OR loan_status='In Grace Period' OR loan_status='Does not meet the credit policy. Status:Fully Paid' AND id in(select id from loan order by random());" , conn)
conn.close()
print("Done Loading SQL Data")



Done Loading SQL Data


In [53]:
risky_loans_raw = df_default
safe_loans_raw = df_fully_paid

#### RENAMING VARIABLES FOR CONSISTENCY ####
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))

#### RANDOMLY SAMPLE 50,000 points from DEFAULTED LOANS AND NOT ####
risky_loans = risky_loans_raw.sample(random_state=1, n=50000)
safe_loans = safe_loans_raw.sample(random_state=1, n=50000)
train_data = risky_loans[:int(len(risky_loans)/2)].append(safe_loans[:int(len(safe_loans)/2)])
validation_data = risky_loans[int(len(risky_loans)/2):].append(safe_loans[int(len(safe_loans)/2):])

print(len(train_data))
print(len(validation_data))

validation_data

# risky_loans[:int(len(risky_loans)/2)]
# train_data




50000
50000


,index,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
20548,113790,3726535,4718929.0,25000.0,25000.0,24975.000000,60 months,15.31%,598.83,C,...,None,None,None,None,None,None,60000.0,None,None,None
26917,157557,1508248,1769878.0,5000.0,5000.0,5000.000000,36 months,14.09%,171.11,B,...,None,None,None,None,None,None,42500.0,None,None,None
35327,54022,30435412,32988574.0,15625.0,15625.0,15625.000000,36 months,16.29%,551.57,D,...,None,None,None,None,None,None,10200.0,None,None,None
9530,28453,8024959,9747089.0,21000.0,21000.0,21000.000000,60 months,22.40%,584.79,E,...,None,None,None,None,None,None,4000.0,None,None,None
17028,89233,5505945,6878162.0,33000.0,33000.0,33000.000000,60 months,21.49%,901.89,E,...,None,None,None,None,None,None,39200.0,None,None,None
29655,174019,1298275,1543116.0,6000.0,6000.0,6000.000000,36 months,7.90%,187.75,A,...,None,None,None,None,None,None,NaN,None,None,None
26325,153627,1554025,1820292.0,8000.0,8000.0,8000.000000,36 months,12.12%,266.18,B,...,None,None,None,None,None,None,5900.0,None,None,None
11398,44718,7388548,9050558.0,5000.0,5000.0,5000.000000,36 months,11.99%,166.05,B,...,None,None,None,None,None,None,14400.0,None,None,None
4168,28439,539020,696116.0,6000.0,6000.0,5975.000000,36 months,11.49%,197.83,B,...,None,None,None,None,None,None,NaN,None,None,None
57617,350292,41950519,44897249.0,35000.0,35000.0,35000.000000,60 months,17.57%,880.61,D,...,None,None,None,None,None,None,28500.0,None,None,None


In [ ]:
#### START TRAINING AND VALIDATION SPLIT ####
# append the training sets together first 25,000 defaults, next 25,000 non-defaults
# data_train = df_def_1.append(df_fp_1)
# data_validation = df_def_2.append(df_fp_2)
# train_data


In [105]:
# set up 
sub_g_labels = ['A1','A2','A3','A4','A5','B1','B2','B3','B4','B5','C1','C2','C3','C4','C5','D1','D2','D3'\
            ,'D4','D5','E1','E2','E3','E4','E5','F1','F2','F3','F4','F5','G1','G2','G3','G4','G5']
grade_v = [(1.0/35 * i) for i in range(1, 36)]
# Dictionary Maps Letter Combo to subgrade
sub_grade_dict = dict(zip(sub_g_labels, range(1, 36)))

print(len(sub_g_labels))
print(len(sub_grade_dict))



# print(sub_grade_dict)
# print(df.axes)
train_data[0]['home_ownership']

35
35


NameError: name 'home_ownership' is not defined

In [159]:
train_data.itertuples()

In [168]:
# X_train = f['sub_grade','loan_amnt']
# need y predictions combine(charged off/default(True(1)) or fully paid(False(0)))
# y_train =[]
# 11th spot in the pandas df if read in all columns
#### INDEXES WITHIN DATA FRAME 
sub_grade, loan_amt, loan_status = 11, 4, 18
avg_loan = 5000


def feature(datum):
    s_grade = 'B1'  # B3 best so far
    loan = avg_loan
    if sub_grade in datum:
        s_grade = datum[sub_grade]
#     if sub_grade not in datum: 
#         print("HERE")
    
    if loan_amt in datum:
        loan = datum[loan_amt]
#     return [sub_grade_dict[datum[sub_grade]], datum[loan_amt]]
    return [sub_grade_dict[s_grade], loan]

default = 0
paid = 0

def prediction_of_y(datum):
    if ((datum[loan_status] == 'Charged Off') or (datum[loan_status] == 'Default') or (datum[loan_status] == 'Late (31-120 days)' ) or (datum[loan_status] == 'Late (16-30 days)') or (datum[loan_status] =='Does not meet the credit policy. Status:Charged Off')):
        return True
    else:
        return False

X_train = [feature(d) for d in train_data.itertuples()]
y_train = [prediction_of_y(d) for d in train_data.itertuples()]
X_test = [feature(d) for d in validation_data.itertuples()]
y_test = [prediction_of_y(d) for d in validation_data.itertuples()]

    
    
    
    
print("Done Predicting")
# Sci-Kit Implementation


Done Predicting


In [124]:
X_test

[[18, 5000],
 [10, 5000],
 [18, 5000],
 [18, 21000.0],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 4800.0],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [21, 5000],
 [18, 5000],
 [14, 5000],
 [18, 16000.0],
 [24, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [7, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [1, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 4500.0],
 [18, 5000],
 [18, 5000],
 [18, 20000.0],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 23700.0],
 [7, 5000],
 [18, 16000.0],
 [18, 21850.0],
 [18, 5000],
 [18, 5000],
 [18, 5000],
 [18, 

In [119]:
rng = np.random.RandomState(1)
clf_dt = DecisionTreeClassifier(random_state=0, max_depth=1)
clf_dt_r = DecisionTreeRegressor(max_depth=6)
clf_boost = AdaBoostRegressor( DecisionTreeRegressor(max_depth=6), n_estimators=400, random_state=rng) 

# Random Forest Trees
clf_random = RandomForestClassifier(max_depth=1, random_state=0, min_samples_split=200, n_estimators=100, n_jobs=-1)
clf_random.fit(X, y)
y_random = clf_random.predict(X_test)
y_random_train = clf_random.predict(X)


X = X_train
y = y_train

clf_dt.fit(X, y)
clf_dt_r.fit(X, y)
print("Started Boosting")
clf_boost.fit(X, y)

y_dt = clf_dt.predict(X_test)
y_dt_r = clf_dt_r.predict(X_test)
y_boost = clf_boost.predict(X)
y_boost_train = clf_boost.predict(X)

print("Decision Tree Training Accuracy: ",  accuracy_score(y_dt, y_test)*100)
print("Random Forest Tree Training Accuracy: ",  accuracy_score(y_random, y_test)*100)
print("Random Forest Tree Training Accuracy: ",  accuracy_score(y_random_train, y)*100)


y_dt_r = []
# print("Decision Tree (Regression) Training Accuracy: ",  accuracy_score(y_dt_r, y_test)*100)
# print("Boosted Decision Tree Training Accuracy: ",  accuracy_score(y_boost, y_test)*100)



Started Boosting
Decision Tree Training Accuracy:  90.308
Random Forest Tree Training Accuracy:  89.1045
Random Forest Tree Training Accuracy:  64.604


In [169]:
#### DATA 200,0000 ####
#### TESTING ON ALL DATA ####      MAX DEPTH 0, 1, 2, N_ESTIMATORS 100-200-500
clf_random = RandomForestClassifier(max_depth=50, random_state=0, min_samples_split=100, n_estimators=100, n_jobs=-1)
clf_random.fit(X, y)
conn = sqlite3.connect(os.path.realpath('') + "/lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 800000);", conn)
# df = pd.read_sql_query("select * from loan", conn)

X_test = [feature(d) for d in df.itertuples()]
y_test = [prediction_of_y(d) for d in df.itertuples()]
y_random = clf_random.predict(X_test)
conn.close()

print("Random Forest Tree Training Accuracy: ",  accuracy_score(y_random, y_test)*100)


#### TESTING ON ALL DATA ####
clf_random = RandomForestClassifier(max_depth=50, random_state=0, min_samples_split=200, n_estimators=100, n_jobs=-1)
clf_random.fit(X, y)
conn = sqlite3.connect(os.path.realpath('') + "/lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 800000);", conn)
# df = pd.read_sql_query("select * from loan", conn)

X_test = [feature(d) for d in df.itertuples()]
y_test = [prediction_of_y(d) for d in df.itertuples()]
y_random = clf_random.predict(X_test)
conn.close()

print("Random Forest Tree Training Accuracy: ",  accuracy_score(y_random, y_test)*100)


#### TESTING ON ALL DATA ####
clf_random = RandomForestClassifier(max_depth=50, random_state=0, min_samples_split=200, n_estimators=500, n_jobs=-1)
clf_random.fit(X, y)
conn = sqlite3.connect(os.path.realpath('') + "/lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 800000);", conn)
# df = pd.read_sql_query("select * from loan", conn)

X_test = [feature(d) for d in df.itertuples()]
y_test = [prediction_of_y(d) for d in df.itertuples()]
y_random = clf_random.predict(X_test)
conn.close()

print("Random Forest Tree Training Accuracy: ",  accuracy_score(y_random, y_test)*100)


#### TESTING ON LINEAR SVC ####
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
clf = LinearSVC(random_state=0)
clf.fit(X, y)
conn = sqlite3.connect(os.path.realpath('') + "/lending-club-loan-data/database.sqlite")
df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 800000);", conn)
# df = pd.read_sql_query("select * from loan", conn)

X_test = [feature(d) for d in df.itertuples()]
y_test = [prediction_of_y(d) for d in df.itertuples()]
y_random = clf.predict(X_test)
conn.close()

print("SVM Training Accuracy: ",  accuracy_score(y_random, y_test)*100)


Random Forest Tree Training Accuracy:  87.833625
Random Forest Tree Training Accuracy:  87.9245
Random Forest Tree Training Accuracy:  87.93375
SVM Training Accuracy:  6.89175


In [172]:
sum(y_random)
y_random[0]

True

In [170]:
len(X_test)

clf_dt = DecisionTreeClassifier(random_state=0, max_depth=6)
clf_dt.fit(X, y)
y_dt = clf_dt.predict(X_test)


n_nodes = clf_dt.tree_.node_count
print(n_nodes)
print("Decision Tree Training Accuracy: ",  accuracy_score(y_random, y_test)*100)


95
Decision Tree Training Accuracy:  6.89175


In [147]:
X

[[8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 4800.0],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [21, 5000],
 [8, 5000],
 [8, 5000],
 [16, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [18, 5000],
 [8, 5000.0],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 25000.0],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [16, 5000],
 [8, 5000],
 [8, 5000],
 [8, 16000.0],
 [8, 5000],
 [8, 35000.0],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [20, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 5000],
 [8, 9375.0],
 [8, 5000],
 [8, 5000],
 [18, 5000],
 [8, 5500.0],
 [8, 